# VKM Dataset Opschoning

**Proces:** Standaardiseer → Kwaliteitscheck → Vul shortdescription → Genereer tags → Export

**Filtering:** Description/content ≤30 karakters worden gemarkeerd (te kort voor TF-IDF/embeddings)

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re

try:
    stopwords.words('dutch')
except LookupError:
    nltk.download('stopwords', quiet=True)

df = pd.read_csv('Uitgebreide_VKM_dataset.csv')
df.drop_duplicates(inplace=True); pd.set_option('display.max_columns', None)
print("Kolommen en dtypes:")
for c in df.columns:
    print(f"  {c}: {df[c].dtype}")
print(f"Dataset geladen: {len(df)} rijen, {len(df.columns)} kolommen")


Kolommen en dtypes:
  id: int64
  name: object
  shortdescription: object
  description: object
  content: object
  studycredit: int64
  location: object
  contact_id: int64
  level: object
  learningoutcomes: object
  Rood: float64
  Groen: float64
  Blauw: float64
  Geel: float64
  module_tags: object
  interests_match_score: float64
  popularity_score: int64
  estimated_difficulty: int64
  available_spots: int64
  start_date: object
Dataset geladen: 211 rijen, 20 kolommen


## We vervangen alle lege/ntb waarden door 'Nog te bepalen'

In [8]:
vooraf_ntb = {}
for col in df.columns:
    if df[col].dtype == 'object':
        vooraf_ntb[col] = (df[col] == 'Nog te bepalen').sum()

for col in df.columns:
    if df[col].dtype == 'object':
        for i in df.index:
            cell_value = str(df.loc[i, col]).strip()
            
            if col == 'module_tags':
                if cell_value in ['[]', "['ntb']", '["ntb"]', 'ntb']:
                    df.loc[i, col] = 'Nog te bepalen'
            else:
                if pd.isna(df.loc[i, col]) or cell_value == '' or cell_value.lower() in ['ntb', 'nog niet bekend', 'nog te formuleren']:
                    df.loc[i, col] = 'Nog te bepalen'

df.drop(['Rood', 'Groen', 'Blauw', 'Geel'], axis=1, inplace=True, errors='ignore')

print("\nNa standaardisatie met nog te bepalen waarden:")
for col in df.columns:
    if df[col].dtype == 'object':
        na_ntb = (df[col] == 'Nog te bepalen').sum()
        verschil = na_ntb - vooraf_ntb.get(col, 0)
        if verschil > 0:
            print(f"  {col}: {na_ntb}")


Na standaardisatie met nog te bepalen waarden:
  shortdescription: 30
  description: 4
  content: 4
  learningoutcomes: 55
  module_tags: 30

  shortdescription: 30
  description: 4
  content: 4
  learningoutcomes: 55
  module_tags: 30


## We checken of description en content lang genoeg zijn (>30 karakters)

In [9]:
print("Vooraf kwaliteitscontrole:")
print(f"  description 'Nog te bepalen': {(df['description'] == 'Nog te bepalen').sum()}")
print(f"  content 'Nog te bepalen': {(df['content'] == 'Nog te bepalen').sum()}")

for i in df.index:
    for col in ['description', 'content']:
        if df.loc[i, col] != 'Nog te bepalen':
            text_len = len(str(df.loc[i, col]).strip())
            if text_len <= 30:
                df.loc[i, col] = 'Nog te bepalen'

print("\nNa kwaliteitscontrole:")
print(f"  description 'Nog te bepalen': {(df['description'] == 'Nog te bepalen').sum()}")
print(f"  content 'Nog te bepalen': {(df['content'] == 'Nog te bepalen').sum()}")

Vooraf kwaliteitscontrole:
  description 'Nog te bepalen': 4
  content 'Nog te bepalen': 4

Na kwaliteitscontrole:
  description 'Nog te bepalen': 7
  content 'Nog te bepalen': 12


## Hier vullen we de shortdescription met de description + content als deze geen Nog te bepalen bevatten

In [10]:
ntb_voor = (df['shortdescription'] == 'Nog te bepalen').sum()
print(f"Vooraf shortdescription 'Nog te bepalen': {ntb_voor}")

for i in df.index:
    if df.loc[i, 'shortdescription'] == 'Nog te bepalen':
        desc = str(df.loc[i, 'description']) if df.loc[i, 'description'] != 'Nog te bepalen' else ''
        cont = str(df.loc[i, 'content']) if df.loc[i, 'content'] != 'Nog te bepalen' else ''
        combined = (desc + ' ' + cont).strip()
        df.loc[i, 'shortdescription'] = combined if combined else 'Nog te bepalen'

ntb_na = (df['shortdescription'] == 'Nog te bepalen').sum()
print(f"Na vullen shortdescription 'Nog te bepalen': {ntb_na}")

Vooraf shortdescription 'Nog te bepalen': 30
Na vullen shortdescription 'Nog te bepalen': 4


## We vullen de module_tags in door belangrijke woorden te stelen uit de shortdescription. We skippen de niet belangrijke woorden!

In [11]:
dutch_stops = set(stopwords.words('dutch'))

ntb_voor = (df['module_tags'] == 'Nog te bepalen').sum()
print(f"Vooraf module_tags 'Nog te bepalen': {ntb_voor}")

aangepaste_rijen = []

for i in df.index:
    if df.loc[i, 'module_tags'] == 'Nog te bepalen':
        aangepaste_rijen.append(i)
        
        text = str(df.loc[i, 'shortdescription'])
        woorden = re.findall(r'\b\w+\b', text.lower())
        tags = [w for w in woorden if w not in dutch_stops and len(w) >= 3 and w not in ['nog', 'bepalen']]
        
        if not tags:
            name_text = str(df.loc[i, 'name'])
            name_woorden = re.findall(r'\b\w+\b', name_text.lower())
            tags = [w for w in name_woorden if w not in dutch_stops and len(w) >= 3 and w not in ['nog', 'bepalen']]
        
        df.loc[i, 'module_tags'] = str(tags) if tags else "['algemeen']"

ntb_na = (df['module_tags'] == 'Nog te bepalen').sum()
print(f"Na vullen module_tags 'Nog te bepalen': {ntb_na}")
print(f"Aantal aangepaste rijen: {len(aangepaste_rijen)}")

Vooraf module_tags 'Nog te bepalen': 30
Na vullen module_tags 'Nog te bepalen': 0
Aantal aangepaste rijen: 30
Na vullen module_tags 'Nog te bepalen': 0
Aantal aangepaste rijen: 30


## Exporten naar een nieuwe .csv bestand voor controle en een print van alle Nog te bepalen velden

In [12]:
output_file = 'Opgeschoonde_VKM_dataset.csv'
df.to_csv(output_file, index=False, encoding='utf-8')

print("Resterende 'Nog te bepalen':")
for col in df.columns:
    if df[col].dtype == 'object':
        ntb_count = (df[col] == 'Nog te bepalen').sum()
        if ntb_count > 0:
            perc = round((ntb_count / len(df)) * 100, 1)
            print(f"  {col}: {ntb_count} ({perc}%)")

Resterende 'Nog te bepalen':
  shortdescription: 4 (1.9%)
  description: 7 (3.3%)
  content: 12 (5.7%)
  learningoutcomes: 55 (26.1%)
